In [5]:
import clickhouse_connect
import polars as pl

In [81]:
client = clickhouse_connect.get_client(host='172.25.0.2', username='default', password='Password123!')

In [82]:
result = client.query("SELECT YEAR, MONTH, MONTH_NAME, STATEFIP, STATEFIP_NAME, COUNTY, METAREA_NAME, EDUC_NAME, HIGRADE_NAME, FTOTVAL, INCTOT FROM cps_00004.cps_00004 LIMIT 1000000")

In [83]:
next(result.named_results())

{'YEAR': 2005,
 'MONTH': 3,
 'MONTH_NAME': 'March',
 'STATEFIP': 48,
 'STATEFIP_NAME': '48',
 'COUNTY': 0,
 'METAREA_NAME': 'Houston-Baytown-Sugar Land, TX',
 'EDUC_NAME': 'Grade 9',
 'HIGRADE_NAME': '',
 'FTOTVAL': 0,
 'INCTOT': 0}

In [84]:
df = pl.from_dicts(result.named_results(), infer_schema_length=400)

In [ ]:
df

In [85]:
df = df.filter(pl.col('INCTOT') != 999999999).filter(pl.col('INCTOT')  != 999999998)

In [86]:
df

YEAR,MONTH,MONTH_NAME,STATEFIP,STATEFIP_NAME,COUNTY,METAREA_NAME,EDUC_NAME,HIGRADE_NAME,FTOTVAL,INCTOT
i64,i64,str,i64,str,i64,str,str,str,i64,i64
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""Grade 9""","""""",0,0
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""Grade 9""","""""",1800,1800
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""Associate's de…","""""",42000,22000
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""High school di…","""""",42000,20000
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""Some college b…","""""",7056,7056
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""High school di…","""""",32000,32000
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""Master's degre…","""""",52288,52288
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""Grades 5 or 6""","""""",8400,8400
2005,3,"""March""",48,"""48""",0,"""Houston-Baytow…","""Bachelor's deg…","""""",8400,0


In [87]:
print(df.group_by(["EDUC_NAME"]).agg(pl.col('INCTOT').mean()).sort('INCTOT') )


shape: (32, 2)
┌───────────────────────────────────┬───────────────┐
│ EDUC_NAME                         ┆ INCTOT        │
│ ---                               ┆ ---           │
│ str                               ┆ f64           │
╞═══════════════════════════════════╪═══════════════╡
│                                   ┆ 2597.249342   │
│ Grade 7                           ┆ 2918.08252    │
│ Grade 1                           ┆ 2921.529358   │
│ Grade 3                           ┆ 3140.377948   │
│ Grade 2                           ┆ 3329.912671   │
│ …                                 ┆ …             │
│ Associate's degree, academic pro… ┆ 33617.450334  │
│ Bachelor's degree                 ┆ 45425.98721   │
│ Master's degree                   ┆ 58583.712619  │
│ Doctorate degree                  ┆ 82560.905947  │
│ Professional school degree        ┆ 101969.509904 │
└───────────────────────────────────┴───────────────┘
